In [2]:
import numpy as np
from random import random
from scipy.optimize import minimize

from qiskit import *
from qiskit.circuit.library.standard_gates import U2Gate
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.algorithms import NumPyEigensolver

# For H1

In [3]:
def hamiltonian_operator(a, b):
    """
    Creates a*I + b*Z pauli sum 
    that will be our Hamiltonian operator.
    
    """
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "I"},
                   {"coeff": {"imag": 0.0, "real": b}, "label": "Z"},
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

In [4]:
a, b = -0.218291, 0.218291
H = hamiltonian_operator(a, b)

In [5]:
exact_result = NumPyEigensolver(H).run()
reference_energy = min(np.real(exact_result.eigenvalues))
print('The exact ground state energy is: {}'.format(reference_energy))

The exact ground state energy is: -0.436582


In [6]:
def quantum_state_preparation(circuit, parameters):
    q = circuit.qregs[0] # q is the quantum register where the info about qubits is stored
    circuit.rx(parameters[0], q[0]) # q[0] is our one and only qubit XD
    circuit.ry(parameters[1], q[0])
    return circuit

In [7]:
def vqe_circuit(parameters, measure):
    """
    Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'Z' stands for Z measurement.
    :return: quantum circuit.
    """
    q = QuantumRegister(1)
    c = ClassicalRegister(1)
    circuit = QuantumCircuit(q, c)

    # quantum state preparation
    circuit = quantum_state_preparation(circuit, parameters)

    # measurement
    if measure == 'Z':
        circuit.measure(q[0], c[0])
    else:
        raise ValueError('Not valid input for measurement: input should be "Z"')

    return circuit


In [8]:
def quantum_module(parameters, measure):
    # measure
    if measure == 'I':
        return 1
    elif measure == 'Z':
        circuit = vqe_circuit(parameters, 'Z')
    else:
        raise ValueError('Not valid input for measurement: input should be "I" or "Z"')
    
    shots = 8192
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    # expectation value estimation from counts
    expectation_value = 0
    for measure_result in counts:
        sign = +1
        if measure_result == '1':
            sign = -1
        expectation_value += sign * counts[measure_result] / shots
        
    return expectation_value

In [9]:
def pauli_operator_to_dict(pauli_operator):
    """
    :param pauli_operator: qiskit's WeightedPauliOperator
    :return: a dict in the desired form.
    """
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}

    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff

    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)

In [10]:
def vqe(parameters):
        
    # quantum_modules
    quantum_module_I = pauli_dict['I'] * quantum_module(parameters, 'I')
    quantum_module_Z = pauli_dict['Z'] * quantum_module(parameters, 'Z')
    
    # summing the measurement results
    classical_adder = quantum_module_I + quantum_module_Z 
    return classical_adder

In [11]:
parameters_array = np.array([np.pi, np.pi])
tol = 1e-3 # tolerance for optimization precision.

vqe_result = minimize(vqe, parameters_array, method="Powell", tol=tol)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))

The exact ground state energy is: -0.436582
The estimated ground state energy from VQE algorithm is: -0.436582


# For H2

In [12]:
def hamiltonian_operator(a, b, c, d, e):
    """
    Creates a*II + b*IZ + c*ZI + d*XX + e*YY pauli sum
    that will be our Hamiltonian operator.
   
    """
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "II"},
                   {"coeff": {"imag": 0.0, "real": b}, "label": "IZ"},
                   {"coeff": {"imag": 0.0, "real": c}, "label": "ZI"},
                   {"coeff": {"imag": 0.0, "real": d}, "label": "XX"},
                   {"coeff": {"imag": 0.0, "real": e}, "label": "YY"}
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

In [13]:
a, b, c, d, e = 5.906709 , 0.218291, -6.125 ,-2.143304 ,-2.143304
H = hamiltonian_operator(a, b, c, d, e)

In [ ]:
exact_result = NumPyEigensolver(H).run()
reference_energy = min(np.real(exact_result.eigenvalues))
print('The exact ground state energy is: {}'.format(reference_energy))

In [ ]:
def quantum_state_preparation(circuit, parameters):
    q = circuit.qregs[0] # q is the quantum register where the info about qubits is stored
    #circuit.rx(parameters[0], q[0]) # q[0] is our one and only qubit XD
    circuit.x(q[0])
    circuit.ry(parameters[0], q[0])
    return circuit

In [ ]:
H_gate = U2Gate(0, np.pi).to_matrix()
print("H_gate:")
print((H_gate * np.sqrt(2)).round(5))

Y_gate = U2Gate(0, np.pi/2).to_matrix()
print("Y_gate:")
print((Y_gate * np.sqrt(2)).round(5))



In [ ]:
def get_from(d: dict, key: str):

    value = 0
    if key in d:
        value = d[key]
    return value

In [ ]:
def vqe_circuit(parameters, measure):
    """Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'Z' stands for Z measurement.
    :return: quantum circuit.
    """
    q = QuantumRegister(2)
    c = ClassicalRegister(2)
    circuit = QuantumCircuit(q, c)
    
    # quantum state preparation
    #circuit = quantum_state_preparation(circuit, parameters)
    #circuit.x(0)
    #circuit.ry(parameters[0], 1)
    q = circuit.qregs[0]
    circuit.x(q[0])
    circuit.ry(parameters[0],q[1])
    circuit.cx(q[1], q[0])
    
    
    
    # measurement
    if measure == 'IZ':
        circuit.u2(0, np.pi, q[0])
        #circuit.z(0)
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'ZI':
        circuit.u2(0, np.pi, q[1])
        #circuit.z(1)
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'XX':
        circuit.u2(0, np.pi, q[0])
        circuit.u2(0, np.pi, q[1])
        #circuit.x(0)
        #circuit.x(1)
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'YY':
        circuit.u2(0, np.pi/2, q[0])
        circuit.u2(0, np.pi/2, q[1])
        #circuit.y(0)
        #circuit.y(1)
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    else:
        raise ValueError('Not valid input for measurement: input should be "X" or "Y" or "Z"')
    
    return circuit


In [ ]:
def quantum_module(parameters, measure):
    # measure
    if measure == 'II':
        return 1
    elif measure == 'IZ':
        circuit = vqe_circuit(parameters, 'IZ')
    elif measure == 'ZI':
        circuit = vqe_circuit(parameters, 'ZI')
    elif measure == 'XX':
        circuit = vqe_circuit(parameters, 'XX')
    elif measure == 'YY':
        circuit = vqe_circuit(parameters, 'YY')
    else:
        raise ValueError('Not valid input for measurement: input should be "I" or "X" or "Z" or "Y"')
   
    shots = 1000
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    # expectation value estimation from counts
    expectation_value = 0
    for measure_result in counts:
        sign = +1
        if measure_result == '1':
            sign = -1
        #expectation_value += sign * counts[measure_result] / shots
        expectation_value = ((get_from(counts, '00')+get_from(counts, '11')) -
                            (get_from(counts,'10')+get_from(counts, '01'))) / shots  
    return expectation_value



In [ ]:
def pauli_operator_to_dict(pauli_operator):
    """
    from WeightedPauliOperator return a dict:
    {I: 0.7, X: 0.6, Z: 0.1, Y: 0.5}.
    :param pauli_operator: qiskit's WeightedPauliOperator
    :return: a dict in the desired form.
    """
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}
    
    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff
    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)

In [ ]:
def vqe(parameters):
       
    # quantum_modules
    quantum_module_II = pauli_dict['II'] * quantum_module(parameters, 'II')
    quantum_module_IZ = pauli_dict['IZ'] * quantum_module(parameters, 'IZ')
    quantum_module_ZI = pauli_dict['ZI'] * quantum_module(parameters, 'ZI')
    quantum_module_XX = pauli_dict['XX'] * quantum_module(parameters, 'XX')
    quantum_module_YY = pauli_dict['YY'] * quantum_module(parameters, 'YY')
   
    # summing the measurement results
    classical_adder = quantum_module_II + quantum_module_IZ+ quantum_module_ZI + quantum_module_XX + quantum_module_YY
   
    return classical_adder

In [ ]:
parameters_array = np.array([np.pi, np.pi])
tol = 1e-3 # tolerance for optimization precision.

vqe_result = minimize(vqe, parameters_array, method="Powell", tol=tol)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))

In [22]:
def hamiltonian_operator(a, b, c, d, e, f, g, h):
    """
    Creates a*III + b*IIZ + c*IZI + d*ZII + e*IXX + f*IYY + g*XXI + h*YYI pauli sum
    that will be our Hamiltonian operator.
   
    """
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "III"},
                   {"coeff": {"imag": 0.0, "real": b}, "label": "IIZ"},
                   {"coeff": {"imag": 0.0, "real": c}, "label": "IZI"},
                   {"coeff": {"imag": 0.0, "real": d}, "label": "ZII"},
                   {"coeff": {"imag": 0.0, "real": e}, "label": "IXX"},
                   {"coeff": {"imag": 0.0, "real": f}, "label": "IYY"},
                   {"coeff": {"imag": 0.0, "real": g}, "label": "XXI"},
                   {"coeff": {"imag": 0.0, "real": h}, "label": "YYI"},
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)



In [23]:
a, b, c, d, e, f, g, h = 15.531709, 0.218291, -6.125, -9.625, -2.143304, -2.143304, -3.913119, -3.913119
H = hamiltonian_operator(a, b, c, d, e, f, g, h)


In [24]:
exact_result = NumPyEigensolver(H).run()
reference_energy = min(np.real(exact_result.eigenvalues))
print('The exact ground state energy is: {}'.format(reference_energy))


The exact ground state energy is: -2.045672287677091


In [30]:
def vqe_circuit(parameters, measure):
    """Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'IIZ' stands for Z0 measurement,  'XXI' stands for X1X2 measurement.
    :return: quantum circuit.
    """
    q = QuantumRegister(3)
    c = ClassicalRegister(3)
    circuit = QuantumCircuit(q, c)
    
#     circuit.x(0)
#     circuit.ry(parameters[0],q[1])
#     circuit.ry(parameters[1],q[2])
#     circuit.cx(q[2], q[0])
#     circuit.cx(q[0], q[1])
#     circuit.barrier(q[0], q[1], q[2])
#     circuit.ry(-parameters[0], q[1])
#     circuit.cx(q[0], q[1])
#     circuit.cx(q[1], q[0])
    
    circuit.x(q[2])
    circuit.ry(parameters[0],q[1])
    circuit.cx(q[1],q[2])
    circuit.cx(q[1],q[0])
    circuit.ry(parameters[1],q[0])
    circuit.cx(q[1],q[0])
    circuit.ry(-parameters[1],q[0])
    circuit.cx(q[0],q[1])
    
        
    
    # measurement
    if measure == 'IIZ':
        circuit.barrier(q[0],q[1],q[2])
        circuit.measure(q[0], c[0])
    elif measure == 'IZI':
        circuit.barrier(q[0],q[1],q[2])
        circuit.measure(q[1], c[1])
    elif measure == 'ZII':
        circuit.barrier(q[0],q[1],q[2])
        circuit.measure(q[2], c[2])
    elif measure == 'IXX':
        circuit.barrier(q[0],q[1],q[2])
        circuit.u(np.pi/2, 0, np.pi, q[0])
        circuit.u(np.pi/2, 0, np.pi, q[1])
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'IYY':
        circuit.barrier(q[0],q[1],q[2])
        circuit.u(np.pi/2, 0, np.pi/2, q[0])
        circuit.u(np.pi/2,0, np.pi/2, q[1])
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'XXI':
        circuit.barrier(q[0],q[1],q[2])
        circuit.u(np.pi/2, 0, np.pi, q[1])
        circuit.u(np.pi/2, 0, np.pi, q[2])
        circuit.measure(q[1], c[1])
        circuit.measure(q[2], c[2])
    elif measure == 'YYI':
        circuit.barrier(q[0],q[1],q[2])
        circuit.u(np.pi/2, 0, np.pi/2, q[1])
        circuit.u(np.pi/2,0, np.pi/2, q[2])
        circuit.measure(q[1], c[1])
        circuit.measure(q[2], c[2])
    else:
        raise ValueError('Not valid input for measurement')
    
    return circuit


SyntaxError: invalid syntax (<ipython-input-30-78b1e008195a>, line 28)

In [31]:
def quantum_module(parameters, measure):
    # measure
    if measure == 'III':
        return 1
    elif measure == 'IIZ':
        circuit = vqe_circuit(parameters, 'IIZ')
    elif measure == 'IZI':
        circuit = vqe_circuit(parameters, 'IZI')
    elif measure == 'ZII':
        circuit = vqe_circuit(parameters, 'ZII')
    elif measure == 'IXX':
        circuit = vqe_circuit(parameters, 'IXX')
    elif measure == 'IYY':
        circuit = vqe_circuit(parameters, 'IYY')
    elif measure == 'XXI':
        circuit = vqe_circuit(parameters, 'XXI')
    elif measure == 'YYI':
        circuit = vqe_circuit(parameters, 'YYI')
    else:
        raise ValueError('Not valid input for measurement: input should be "III" or "IIZ" or "IZI" or "ZII" or "IXX" or "IYY" or "XXI" or "YYI"')
   
    shots = 10000
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    # expectation value estimation from counts
    expectation_value = 0
    for measure_result in counts:
        sign = -1
        if (measure_result == '000') or (measure_result == '011')  or (measure_result == '101')  or (measure_result == '110'):
             sign = +1
        expectation_value += sign * counts[measure_result] / shots
        
    return expectation_value

In [32]:
def pauli_operator_to_dict(pauli_operator):
    """
    from WeightedPauliOperator return a dict:
    {III: 15.531709, IIZ: 0.218291, IZI: -6.125, ZII: -9.625, IXX: -2.143304, IYY: -2.143304, XXI: -3.913119, YYI: -3.913119}.
    :param pauli_operator: qiskit's WeightedPauliOperator
    :return: a dict in the desired form.
    """
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}
    
    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff
    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)


In [33]:
def vqe(parameters):
       
    # quantum_modules
    quantum_module_III = pauli_dict['III'] * quantum_module(parameters, 'III')
    quantum_module_IIZ = pauli_dict['IIZ'] * quantum_module(parameters, 'IIZ')
    quantum_module_IZI = pauli_dict['IZI'] * quantum_module(parameters, 'IZI')
    quantum_module_ZII = pauli_dict['ZII'] * quantum_module(parameters, 'ZII')
    quantum_module_IXX = pauli_dict['IXX'] * quantum_module(parameters, 'IXX')
    quantum_module_IYY = pauli_dict['IYY'] * quantum_module(parameters, 'IYY')
    quantum_module_XXI = pauli_dict['XXI'] * quantum_module(parameters, 'XXI')
    quantum_module_YYI = pauli_dict['YYI'] * quantum_module(parameters, 'YYI')
   
    # summing the measurement results
    classical_adder = quantum_module_III + quantum_module_IIZ + quantum_module_IZI + quantum_module_ZII + quantum_module_IXX  + quantum_module_IYY + quantum_module_XXI + quantum_module_YYI
    return classical_adder



In [34]:
parameters_array = np.array([np.pi/2,np.pi/2])
tol = 1e-3 # tolerance for optimization precision.
a= {"maxiter" : 500}
vqe_result = minimize(vqe, parameters_array, method="COBYLA", tol=tol,options = a)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))
print("\nThe optimal parameter theta is : {} ".format(vqe_result.x))


The exact ground state energy is: -2.045672287677091
The estimated ground state energy from VQE algorithm is: -1.9866647318000004

The optimal parameter theta is : [0.38193092 0.30958056] 


In [2]:
def hamiltonian_operator(a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w):
    """Creates a*IIIIIIII + b*IIIIIIIZ + c*IIIIIIZI + d*IIIIIZII + e*IIIIZIII + f*IIIZIIII + g*IIZIIIII + h*IZIIIIII+ i*ZIIIIIII + j*IIIIIIXX + k*IIIIIIYY+ l*IIIIIXXI + m*IIIIIYYI + n*IIIIXXII + o*IIIIYYII + p*IIIXXIII + q*IIIYYIII + r*IIXXIIII + s*IIYYIIII + t*IXXIIIII + u*IYYIIIII + v*XXIIIIII + w*YYIIIIII pauli sum
    that will be our Hamiltonian operator.
   
    """
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "IIIIIIII"},
                   {"coeff": {"imag": 0.0, "real": b}, "label": "IIIIIIIZ"},
                   {"coeff": {"imag": 0.0, "real": c}, "label": "IIIIIIZI"},
                   {"coeff": {"imag": 0.0, "real": d}, "label": "IIIIIZII"},
                   {"coeff": {"imag": 0.0, "real": e}, "label": "IIIIZIII"},
                   {"coeff": {"imag": 0.0, "real": f}, "label": "IIIZIIII"},
                   {"coeff": {"imag": 0.0, "real": g}, "label": "IIZIIIII"},
                   {"coeff": {"imag": 0.0, "real": h}, "label": "IZIIIIII"},
                   {"coeff": {"imag": 0.0, "real": i}, "label": "ZIIIIIII"},
                   {"coeff": {"imag": 0.0, "real": j}, "label": "IIIIIIXX"},
                   {"coeff": {"imag": 0.0, "real": k}, "label": "IIIIIIYY"},
                   {"coeff": {"imag": 0.0, "real": l}, "label": "IIIIIXXI"},
                   {"coeff": {"imag": 0.0, "real": m}, "label": "IIIIIYYI"},
                   {"coeff": {"imag": 0.0, "real": n}, "label": "IIIIXXII"},
                   {"coeff": {"imag": 0.0, "real": o}, "label": "IIIIYYII"},
                   {"coeff": {"imag": 0.0, "real": p}, "label": "IIIXXIII"},
                   {"coeff": {"imag": 0.0, "real": q}, "label": "IIIYYIII"},
                   {"coeff": {"imag": 0.0, "real": r}, "label": "IIXXIIII"},
                   {"coeff": {"imag": 0.0, "real": s}, "label": "IIYYIIII"},
                   {"coeff": {"imag": 0.0, "real": t}, "label": "IXXIIIII"},
                   {"coeff": {"imag": 0.0, "real": u}, "label": "IYYIIIII"},
                   {"coeff": {"imag": 0.0, "real": v}, "label": "XXIIIIII"},
                   {"coeff": {"imag": 0.0, "real": w}, "label": "YYIIIIII"},
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)


In [3]:
a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w = 116.156709, 0.218219, -6.125, -9.625, -13.125, -16.625, -20.125, -23.625, -27.125, -2.143304, -2.143304, -3.91311,-3.91311, -5.670648, -5.670648, -7.4246, -7.4246, -9.177077, -9.177077, -10.928746, -10.928746, -12.679954, -12.679954
H = hamiltonian_operator(a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w) 

/home/garvit/anaconda3/lib/python3.8/site-packages/qiskit/aqua/operators/legacy/weighted_pauli_operator.py:582: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  return cls(paulis=paulis)


In [4]:
exact_result = NumPyEigensolver(H).run()
reference_energy = min(np.real(exact_result.eigenvalues))
print('The exact ground state energy is: {}'.format(reference_energy))

The exact ground state energy is: -2.214992336323061


/home/garvit/anaconda3/lib/python3.8/site-packages/qiskit/aqua/algorithms/eigen_solvers/eigen_solver.py:36: DeprecationWarning: The package qiskit.aqua.algorithms.eigen_solvers is deprecated. It was moved/refactored to qiskit.algorithms.eigen_solvers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.algorithms.eigen_solvers',


In [7]:
def vqe_circuit(parameters, measure):
    """Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'IIZ' stands for Z0 measurement,  'XXI' stands for X1X2 measurement.
    :return: quantum circuit.
    """
#     q = QuantumRegister(8)
#     c = ClassicalRegister(8)
    circuit = QuantumCircuit(8, 8)
    
#     circuit.x(0)
#     circuit.ry(parameters[0],q[1])
#     circuit.ry(parameters[1],q[2])
#     circuit.cx(q[2], q[0])
#     circuit.cx(q[0], q[1])
#     circuit.barrier(q[0], q[1], q[2])
#     circuit.ry(-parameters[0], q[1])
#     circuit.cx(q[0], q[1])
#     circuit.cx(q[1], q[0])
    
    circuit.x(7)
    circuit.ry(parameters[0],6)
    circuit.cx(6,7)
    circuit.cx(6,5)
    circuit.ry(parameters[1],5)
    circuit.cx(6,5)
    circuit.ry(-parameters[1],5)
    circuit.cx(5,6)
    circuit.cx(5,4)
    circuit.ry(parameters[2],4)
    circuit.cx(5,4)
    circuit.ry(-parameters[2],4)
    circuit.cx(4,5)
    circuit.cx(4,3)
    circuit.ry(parameters[3],3)
    circuit.cx(4,3)
    circuit.ry(-parameters[3],3)
    circuit.cx(3,4)
    circuit.cx(3,2)
    circuit.ry(parameters[4],2)
    circuit.cx(3,2)
    circuit.ry(-parameters[4],2)
    circuit.cx(2,3)
    circuit.cx(2,1)
    circuit.ry(parameters[5],1)
    circuit.cx(2,1)
    circuit.ry(-parameters[5],1)
    circuit.cx(1,2)
    circuit.cx(1,0)
    circuit.ry(parameters[6],0)
    circuit.cx(1,0)
    circuit.ry(-parameters[6],0)
    circuit.cx(0,1)
    
        
    
    # measurement
    if measure == 'IIIIIIIZ':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.measure(0,0)
    elif measure == 'IIIIIIZI':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.measure(1,1)
    elif measure == 'IIIIIZII':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.measure(2,2)
    elif measure == 'IIIIIIXX':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi, 0)
        circuit.u(np.pi/2, 0, np.pi, 1)
        circuit.measure(0,0)
        circuit.measure(1,1)
    elif measure == 'IIIIIIYY':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi/2, 0)
        circuit.u(np.pi/2,0, np.pi/2, 1)
        circuit.measure(0,0)
        circuit.measure(1,1)
    elif measure == 'IIIIIIXXI':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi, 1)
        circuit.u(np.pi/2, 0, np.pi, 2)
        circuit.measure(1,1)
        circuit.measure(2,2)
    elif measure == 'IIIIIIYYI':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi/2, 1)
        circuit.u(np.pi/2,0, np.pi/2, 2)
        circuit.measure(1,1)
        circuit.measure(2,2)
    elif measure == 'IIIIXXII':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi, 2)
        circuit.u(np.pi/2, 0, np.pi, 3)
        circuit.measure(2,2)
        circuit.measure(3,3)
    elif measure == 'IIIIYYII':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi/2, 2)
        circuit.u(np.pi/2, 0, np.pi/2, 3)
        circuit.measure(2,2)
        circuit.measure(3,3)
    elif measure == 'IIIXXIII':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi, 3)
        circuit.u(np.pi/2, 0, np.pi, 4)
        circuit.measure(3,3)
        circuit.measure(4,4)
    elif measure == 'IIIYYIII':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi/2, 3)
        circuit.u(np.pi/2, 0, np.pi/2, 4)
        circuit.measure(3,3)
        circuit.measure(4,4)
    elif measure == 'IIXXIIII':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi, 4)
        circuit.u(np.pi/2, 0, np.pi, 5)
        circuit.measure(4,4)
        circuit.measure(5,5)
    elif measure == 'IIYYIIII':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi/2, 4)
        circuit.u(np.pi/2, 0, np.pi/2, 5)
        circuit.measure(4,4)
        circuit.measure(5,5)
    elif measure == 'IXXIIIII':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi, 5)
        circuit.u(np.pi/2, 0, np.pi, 6)
        circuit.measure(5,5)
        circuit.measure(6,6)
    elif measure == 'IYYIIIII':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi/2, 5)
        circuit.u(np.pi/2, 0, np.pi/2, 6)
        circuit.measure(5,5)
        circuit.measure(6,6)
    elif measure == 'XXIIIIII':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi, 6)
        circuit.u(np.pi/2, 0, np.pi, 7)
        circuit.measure(6,6)
        circuit.measure(7,7)
    elif measure == 'YYIIIIII':
        circuit.barrier(0,1,2,3,4,5,6,7)
        circuit.u(np.pi/2, 0, np.pi/2, 6)
        circuit.u(np.pi/2, 0, np.pi/2, 7)
        circuit.measure(6,6)
        circuit.measure(7,7)
    else:
        raise ValueError('Not valid input for measurement')
    
    return circuit

In [ ]:
def quantum_module(parameters, measure):
    # measure
    if measure == 'IIIIIIII':
        return 1
    elif measure == 'IIIIIIIZ':
        circuit = vqe_circuit(parameters, 'IIIIIIIZ')
    elif measure == 'IIIIIIZI':
        circuit = vqe_circuit(parameters, 'IIIIIIZI')
    elif measure == 'IIIIIZII':
        circuit = vqe_circuit(parameters, 'IIIIIZII')
    elif measure == 'IIIIZIII':
        circuit = vqe_circuit(parameters, 'IIIIZIII')
    elif measure == 'IIIZIIII':
        circuit = vqe_circuit(parameters, 'IIIZIIII')
    elif measure == 'IIZIIIII':
        circuit = vqe_circuit(parameters, 'IIZIIIII')
    elif measure == 'IZIIIIII':
        circuit = vqe_circuit(parameters, 'IZIIIIII')
    elif measure == 'ZIIIIIII':
        circuit = vqe_circuit(parameters, 'ZIIIIIII')
    elif measure == 'IIIIIIXX':
        circuit = vqe_circuit(parameters, 'IIIIIIXX')
    elif measure == 'IIIIIIYY':
        circuit = vqe_circuit(parameters, 'IIIIIIYY')
    elif measure == 'IIIIIXXI':
        circuit = vqe_circuit(parameters, 'IIIIIXXI')
    elif measure == 'IIIIIYYI':
        circuit = vqe_circuit(parameters, 'IIIIIYYI')
    elif measure == 'IIIIXXII':
        circuit = vqe_circuit(parameters, 'IIIIXXII')
    elif measure == 'IIIIYYII':
        circuit = vqe_circuit(parameters, 'IIIIYYII')
    elif measure == 'IIIXXIII':
        circuit = vqe_circuit(parameters, 'IIIXXIII')
    elif measure == 'IIIYYIII':
        circuit = vqe_circuit(parameters, 'IIIYYIII')
    elif measure == 'IIXXIIII':
        circuit = vqe_circuit(parameters, 'IIXXIIII')
    elif measure == 'IIYYIIII':
        circuit = vqe_circuit(parameters, 'IIYYIIII')
    elif measure == 'IXXIIIII':
        circuit = vqe_circuit(parameters, 'IXXIIIII')
    elif measure == 'IYYIIIII':
        circuit = vqe_circuit(parameters, 'IYYIIIII')
    elif measure == 'XXIIIIII':
        circuit = vqe_circuit(parameters, 'XXIIIIII')
    elif measure == 'YYIIIIII':
        circuit = vqe_circuit(parameters, 'YYIIIIII')
    else:
        raise ValueError('Not valid input for measurement: input should be "III" or "IIZ" or "IZI" or "ZII" or "IXX" or "IYY" or "XXI" or "YYI"')
   
    shots = 1000
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    # expectation value estimation from counts
    expectation_value = 0
    for measure_result in counts:
        sign = -1
        if (measure_result == '000') or (measure_result == '011')  or (measure_result == '101')  or (measure_result == '110'):
             sign = +1
        expectation_value += sign * counts[measure_result] / shots
        
    return expectation_value

In [ ]:
def pauli_operator_to_dict(pauli_operator):
    """
    from WeightedPauliOperator return a dict:
    {III: 15.531709, IIZ: 0.218291, IZI: -6.125, ZII: -9.625, IXX: -2.143304, IYY: -2.143304, XXI: -3.913119, YYI: -3.913119}.
    :param pauli_operator: qiskit's WeightedPauliOperator
    :return: a dict in the desired form.
    """
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}
    
    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff
    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)


In [ ]:
def vqe(parameters):
       
    # quantum_modules
    quantum_module_IIIIIIII = pauli_dict['IIIIIIII'] * quantum_module(parameters, 'IIIIIIII')
    quantum_module_IIIIIIIZ = pauli_dict['IIIIIIIZ'] * quantum_module(parameters, 'IIIIIIIZ')
    quantum_module_IIIIIIZI = pauli_dict['IIIIIIZI'] * quantum_module(parameters, 'IIIIIIZI')
    quantum_module_IIIIIZII = pauli_dict['IIIIIZII'] * quantum_module(parameters, 'IIIIIZII')
    quantum_module_IIIIZIII = pauli_dict['IIIIZIII'] * quantum_module(parameters, 'IIIIZIII')
    quantum_module_IIIZIIII = pauli_dict['IIIZIIII'] * quantum_module(parameters, 'IIIZIIII')
    quantum_module_IIZIIIII = pauli_dict['IIZIIIII'] * quantum_module(parameters, 'IIZIIIII')
    quantum_module_IZIIIIII = pauli_dict['IZIIIIII'] * quantum_module(parameters, 'IZIIIIII')
    quantum_module_ZIIIIIII = pauli_dict['ZIIIIIII'] * quantum_module(parameters, 'ZIIIIIII')
    quantum_module_IIIIIIXX = pauli_dict['IIIIIIXX'] * quantum_module(parameters, 'IIIIIIXX')
    quantum_module_IIIIIIYY = pauli_dict['IIIIIIYY'] * quantum_module(parameters, 'IIIIIIYY')
    quantum_module_IIIIIXXI = pauli_dict['IIIIIXXI'] * quantum_module(parameters, 'IIIIIXXI')
    quantum_module_IIIIIYYI = pauli_dict['IIIIIYYI'] * quantum_module(parameters, 'IIIIIYYI')
    quantum_module_IIIIXXII = pauli_dict['IIIIXXII'] * quantum_module(parameters, 'IIIIXXII')
    quantum_module_IIIIYYII = pauli_dict['IIIIYYII'] * quantum_module(parameters, 'IIIIYYII')
    quantum_module_IIIXXIII = pauli_dict['IIIXXIII'] * quantum_module(parameters, 'IIIXXIII')
    quantum_module_IIIYYIII = pauli_dict['IIIYYIII'] * quantum_module(parameters, 'IIIYYIII')
    quantum_module_IIXXIIII = pauli_dict['IIXXIIII'] * quantum_module(parameters, 'IIXXIIII')
    quantum_module_IIYYIIII = pauli_dict['IIYYIIII'] * quantum_module(parameters, 'IIYYIIII')
    quantum_module_IXXIIIII = pauli_dict['IXXIIIII'] * quantum_module(parameters, 'IXXIIIII')
    quantum_module_IYYIIIII = pauli_dict['IYYIIIII'] * quantum_module(parameters, 'IYYIIIII')
    quantum_module_XXIIIIII = pauli_dict['XXIIIIII'] * quantum_module(parameters, 'XXIIIIII')
    quantum_module_YYIIIIII = pauli_dict['YYIIIIII'] * quantum_module(parameters, 'YYIIIIII')
   
    # summing the measurement results
    classical_adder = quantum_module_IIIIIIII + quantum_module_IIIIIIIZ + quantum_module_IIIIIIZI + quantum_module_IIIIIZII +quantum_module_IIIIZIII + quantum_module_IIIZIIII + quantum_module_IIZIIIII + quantum_module_IZIIIIII +quantum_module_ZIIIIIII + quantum_module_IIIIIIXX  + quantum_module_IIIIIIYY + quantum_module_IIIIIXXI + quantum_module_IIIIIYYI + quantum_module_IIIIXXII + quantum_module_IIIIYYII + quantum_module_IIIXXIII + quantum_module_IIIYYIII + quantum_module_IIXXIIII + quantum_module_IIYYIIII + quantum_module_IXXIIIII + quantum_module_IYYIIIII + quantum_module_XXIIIIII + quantum_module_YYIIIIII 
    return classical_adder


In [ ]:
parameters_array = np.array([np.pi/2,np.pi/2,np.pi/2,np.pi/2,np.pi/2,np.pi/2])
tol = 1e-3 # tolerance for optimization precision.
a= {"maxiter" : 100}
vqe_result = minimize(vqe, parameters_array, method="COBYLA", tol=tol,options = a)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))
print("\nThe optimal parameter theta is : {} ".format(vqe_result.x))
